### Install the package we need.

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

### Connect to the drive, to save the information to the local.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import the package we need.

In [ ]:
# Web Server
from flask import Flask, request, abort, jsonify
import json
# Generate the outside url
from flask_ngrok import run_with_ngrok
from linebot import (
    LineBotApi, 
    WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
import config
import os 

In [ ]:
# connect to the line bot api and the handler
line_bot_api = LineBotApi(config.line_bot_api)
handler = WebhookHandler(config.handler)

### load the story with the Excel

In [ ]:
import pandas as pd

plot_content = pd.read_excel("./drive/MyDrive/leftover/bearbear.xlsx")

### Import the package

In [ ]:
from linebot.models import (
    ImagemapSendMessage,
    TextSendMessage,
    ImageSendMessage,
    LocationSendMessage,
    FlexSendMessage,
    VideoSendMessage,
    StickerSendMessage,
    AudioSendMessage
)
from linebot.models.template import (
    ButtonsTemplate,
    CarouselTemplate,
    ConfirmTemplate,
    ImageCarouselTemplate
)
from linebot.models.template import *

In [ ]:
def detect_json_array_to_new_message_array(jsonObjectArray):
    
    # jsonObject = json.loads(jsonObjectString)
    returnArray = []


    # 讀取其用來判斷的元件
    for jsonObject in jsonObjectArray:
      message_type = jsonObject.get('type')
          
      # 轉換
      if message_type == 'text':
          returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
      elif message_type == 'imagemap':
          returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
      elif message_type == 'template':
          returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
      elif message_type == 'image':
          returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
      elif message_type == 'sticker':
          returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
      elif message_type == 'audio':
          returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
      elif message_type == 'location':
          returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
      elif message_type == 'flex':
          returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
      elif message_type == 'video':
          returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    
          
    return returnArray

### Find the repliance from the Excel, then turn them into the message.

In [ ]:
from numpy import NaN
import math
import json

def drama_execl_to_json(user_input_keyword):
  result = plot_content[plot_content['keyword']==user_input_keyword]
  result_dict=result.to_dict()
  for field in result_dict.keys():
    for key in result_dict[field].keys():
      result_dict[field]= result_dict[field][key]
  
  reply_json_array=[]
  combin_json_array=['reply_message1','reply_message2','reply_message3','reply_message4','reply_message5']

  for ele in combin_json_array:
    if pd.isna(result_dict[ele]) is False:
      print(result_dict[ele])
      reply_json_array.append(json.loads(result_dict[ele]))
      print(reply_json_array)

  if pd.isna(result_dict['choice_button']) is False:
    reply_json_array[len(reply_json_array)-1].update(json.loads(result_dict['choice_button']))

  reply_message_array = detect_json_array_to_new_message_array(reply_json_array)
  return reply_message_array

### start the server

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
# 啟動server對外接口，使Line能丟消息進來
@app.route("/callback", methods=['POST'])

def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    with open('./drive/MyDrive/leftover/20230328/ai-event.txt', 'a') as testwritefile:
      testwritefile.write(body)
      testwritefile.write('\n')
    # 記錄用戶log
    with open('./drive/MyDrive/leftover/20230328/ai-event.log', 'a') as testwritefile:
      testwritefile.write(body)
      testwritefile.write('\n')

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息


In [ ]:
from linebot.models.events import (
    FollowEvent,
    MessageEvent
)
from linebot.models import(
    TextSendMessage, 
    ImageSendMessage,
    ImageMessage,
    VideoMessage,
    AudioMessage,
    AudioSendMessage
)

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("./drive/MyDrive/leftover/20230328/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    

    # # 回覆文字消息與圖片消息
    # line_bot_api.reply_message(
    #     event.reply_token,
    #     TextSendMessage('安安，我們成功成為好友了！')
    # )

In [ ]:
from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage,TextMessage
)

@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):

    if len(drama_execl_to_json(event.message.text)) != 0 :
      line_bot_api.reply_message(
        event.reply_token,
        drama_execl_to_json(event.message.text)
      )
    else:
      line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage('此物件沒有劇情設計')
      )


若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。



In [ ]:
from linebot.models import(
    TextSendMessage, ImageMessage,VideoMessage,AudioMessage,AudioSendMessage
)

# 告知handler，當收到消息事件，且消息為圖片消息時，作下列的方法
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):

    # 麻煩line_bot_api回應用戶消息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id + '\n' + '你好啊！你的照片被我拿走了')
    )
    
    # 麻煩line_bot_api跟line索取該消息的多媒體訊息
    # 存回Colab的臨時電腦內
    message_content = line_bot_api.get_message_content(event.message.id)
    # with open(os.path.join('/path/to/Documents',completeName), "w") as file1:
    #     toFile = raw_input("Write what you want into the field")
    #     file1.write(toFile)
    with open('./drive/MyDrive/leftover/20230328/'+event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)


若收到語音消息時，

先回覆用戶文字消息，並從Line上將照片拿回。


In [ ]:
from linebot.models import(
    TextSendMessage, ImageMessage,VideoMessage,AudioMessage,AudioSendMessage
)

# 告知handler，當收到消息事件，且消息為圖片消息時，作下列的方法
@handler.add(MessageEvent, message=AudioMessage)
def handle_audio_message(event):

    # 麻煩line_bot_api回應用戶消息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Audio has Upload'+ ' ' + event.message.id))
    
    # 麻煩line_bot_api跟line索取該消息的多媒體訊息
    # 存回Colab的臨時電腦內
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./drive/MyDrive/leftover/20230328/' + event.message.id+'.mp3', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

若收到影片消息時，

先回覆用戶文字消息，並從Line上將影片拿回。

In [ ]:
from linebot.models import(
    TextSendMessage, ImageMessage,VideoMessage,AudioMessage,AudioSendMessage
)

# 告知handler，當收到消息事件，且消息為圖片消息時，作下列的方法
@handler.add(MessageEvent, message=VideoMessage)
def handle_video_message(event):

    # 麻煩line_bot_api回應用戶消息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Video has Upload'+ ' ' + event.message.id))
    
    # 麻煩line_bot_api跟line索取該消息的多媒體訊息
    # 存回Colab的臨時電腦內
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./drive/MyDrive/leftover/20230328/' + event.message.id+'.mp4', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
if __name__ == '__main__':
    app.run()